This file serves to test the python i would further use in the snakefile without running the whole pipeline
## Test 1 : read results and create a table for datapane

In [43]:
import os
import pandas as pd
import numpy as np
from glob import glob
import altair as alt
import csv
dirname = input('What\'s the folder name honey')
path = '/mnt/R60-11/Bacterio/metagenomique_NextSeq/' + dirname + '/'

In [44]:
with open(path + 'SampleSheet.csv', 'r') as csvfile:
    lines = csvfile.readlines()
sections = [lines.index(line) for line in lines if "[" in line]
SampleSheet = lines[sections[3]+2:]
header = lines[sections[3]+1].split(sep=",")
SampleSheet = pd.DataFrame([line.split(",") for line in SampleSheet], columns = header)[['Sample_ID','Description']].set_index('Sample_ID')
SampleSheet


,Description
Sample_ID,
1,11D
2,12D
3,13D
4,kitomeD
5,11C
6,12C
7,13C
8,kitomeC
9,TPOS


In [45]:
path = '/mnt/R60-11/Bacterio/metagenomique_NextSeq/' + dirname + '/results/temp/'

In [49]:
D = pd.read_table(path + 'Nreads_raw.txt' ,sep=",",names=['raw'],index_col=0)
D = D.join(pd.read_table(path + 'Nreads_qc.txt' ,sep=",",names=['quality'],index_col=0))
D = D.join(pd.read_table(path + 'Nreads_nothumain_1.txt' ,sep=",",names=['human1'],index_col=0))
D = D.join(pd.read_table(path + 'Nreads_nothumain_2.txt' ,sep=",",names=['human2'],index_col=0))
D['nonHuman'] = D["human1"] - D["human2"]
D['percent'] = D['nonHuman'] / D['raw'] * 100
D = D.drop(labels="Undetermined", axis=0)
D =D.join(SampleSheet)
D = D[['Description','raw','quality','nonHuman','percent']].sort_index()
D.index = D.index.astype(int)
D = D[['Description','raw','quality','nonHuman','percent']].sort_index()
D['Sample'] = D.index
D[['Sample','Description','raw','quality','nonHuman','percent']]

,Sample,Description,raw,quality,nonHuman,percent
1,1,11D,1821360,1148155,6492,0.356437
2,2,12D,2148734,1369402,4471,0.208076
3,3,13D,2203017,1477499,16872,0.765859
4,4,kitomeD,263174,99032,28690,10.901533
5,5,11C,1792333,1234955,2049,0.114320
6,6,12C,6942381,5324485,41338,0.595444
7,7,13C,5362957,4059166,41276,0.769650
8,8,kitomeC,7662826,5455151,607278,7.924987
9,9,TPOS,1455065,1220981,1220015,83.846082
10,10,TENV,446,167,117,26.233184


In [7]:
D1 = pd.melt(D[['Sample','raw','nonHuman']], id_vars=['Sample'], value_vars=['raw','nonHuman'], var_name='steps', value_name='reads')
D1.head()

,Sample,steps,reads
0,1,raw,3431382
1,2,raw,972512
2,3,raw,54706
3,4,raw,33124
4,5,raw,1290077


In [15]:
## Plot t
alt.Chart(D1).mark_line().encode(
            x='Sample:N',
            y='reads:Q',
            column='steps:N',
            tooltip = [alt.Tooltip('reads:Q'),alt.Tooltip('Sample:N')]
        ).properties(width=1200,
                    height=300
        ).configure_axis(labelFontSize=15,
                         titleFontSize=20
        )

alt.Chart(...)

## Test 2 : read kraken results and create a table for datapane

In [50]:
import sys 
import os
import argparse
import numpy as np
import pandas as pd
from collections import defaultdict

In [54]:
path = '/mnt/R60-11/Bacterio/metagenomique_NextSeq/' + dirname + '/results/03_not_human/report/'

In [55]:
def input_dir(dir):
	files=os.listdir(dir)
	report=[]
	for f in files:
		fipath=os.path.join(dir,f)
		report.append(fipath)	
	return (report)

In [57]:
filelist=input_dir(path)

In [58]:
alldf = []
for file in filelist:
    if 'Undetermined' in file:
        continue
    out = pd.read_csv(file, sep="\t", names=['percent','reads_root','reads_assigned','rank', 'taxon_id','name'])
    out = out[out["rank"] == "S"]
    out["name"] = out["name"].str.strip()
    out["Sample"] = file.split("/")[-1].split(".")[0]
    alldf.append(out)
    superdf = pd.concat(alldf)
    table = superdf[["name","reads_root", "Sample"]]
    table = table.pivot_table(index="name", columns="Sample", values="reads_root")
    table = table.drop(labels="Homo sapiens", axis=0)
    table = table.fillna(0)
    table = table[table.columns.astype(int).sort_values().astype(str)]
    table.loc[:,'Total'] = table.sum(axis=1)
    table = table[table["Total"] > 100]

In [61]:
table

Sample,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Total
name,,,,,,,,,,,,,,,
Actinomyces oris,0.0,2.0,1.0,1.0,0.0,3.0,4.0,629.0,0.0,0.0,0.0,0.0,0.0,0.0,640.0
Alcanivorax sp. N3-2A,0.0,0.0,0.0,0.0,2.0,10.0,6.0,4545.0,1.0,0.0,0.0,0.0,0.0,0.0,4564.0
Anoxybacillus flavithermus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,326.0,0.0,0.0,0.0,0.0,0.0,0.0,326.0
Atopobium parvulum,26.0,24.0,69.0,2.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,159.0
Bacillus cereus,1.0,0.0,0.0,2.0,5.0,122.0,62.0,25.0,364.0,0.0,0.0,1.0,0.0,1.0,583.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Streptococcus sp. oral taxon 431,116.0,51.0,250.0,1.0,0.0,0.0,0.0,1470.0,0.0,0.0,0.0,2.0,0.0,0.0,1890.0
Streptococcus viridans,23.0,14.0,88.0,0.0,0.0,0.0,0.0,221.0,0.0,0.0,0.0,0.0,0.0,0.0,346.0
Thermoanaerobacterium thermosaccharolyticum,0.0,0.0,2.0,0.0,0.0,1.0,1.0,219.0,0.0,0.0,0.0,0.0,0.0,0.0,223.0
